In [ ]:
!conda install -c intel scikit-learn

# Training a sentiment analysis classifier based on supervised machine learning algorithms

In [ ]:
import string

import pymongo

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import ConfusionMatrixDisplay, precision_score, recall_score, f1_score

import pandas as pd

In [ ]:
stop_words = set(stopwords.words('spanish'))

In [ ]:
# MongoDB connection data
HOST = 'localhost'
PORT = '27017'
DB_NAME = 'trending'

### Connecting to database

In [ ]:
db = None
try:
    db_client = pymongo.MongoClient(HOST + ':' + PORT)
    db = db_client[DB_NAME]
    print('Conecction established successfully!')
except pymongo.errors.ConnectionFailure:
    print('Connection cannot be established')

In [ ]:
HISTORY_COL = db['history']

### Getting all texts and removing duplicates

In [ ]:
tweets = HISTORY_COL.find({}, {'_id': 0, 'full_text': 1})

In [ ]:
tweets_df =  pd.DataFrame(list(tweets))

In [ ]:
tweets_df.shape

In [ ]:
tweets_df['full_text'] = tweets_df['full_text'].str.replace('RT ', '')

In [ ]:
tweets_df.drop_duplicates(inplace = True)

In [ ]:
tweets_df.shape

In [ ]:
tweets_df['sentiment'] = None

In [ ]:
tweets_df.head()

### Labeling

In [ ]:
tweets_df.to_csv('./data/medellin_tweets.csv', index = False, sep = '|')

<br />
<span style="color: red;">Labeling...</span>
<br /><br />

In [ ]:
tweets_df = pd.read_csv('./data/medellin_tweets_labeled.csv', sep = ',')

In [ ]:
tweets_df.head()

In [ ]:
tweets_df['sentiment'].value_counts(dropna = False, normalize = True)

### Leaving out unlabeled texts, this data is not useful for training or validating a supervised model

In [ ]:
tweets_labeled_df = tweets_df.loc[tweets_df['sentiment'].notnull()]

In [ ]:
tweets_labeled_df.shape

In [ ]:
tweets_nolabeled_df = tweets_df.loc[tweets_df['sentiment'].isnull()]

In [ ]:
tweets_nolabeled_df.shape

### Vectorizing text using BOW

<img src="./imgs/bow.png" style="height: 250px;">

In [ ]:
def tokenizer(text):
    tt = TweetTokenizer()
    return tt.tokenize(text)

In [ ]:
vectorizer = CountVectorizer(tokenizer = tokenizer, stop_words = stop_words)

In [ ]:
X = vectorizer.fit_transform(tweets_labeled_df['full_text'])

In [ ]:
vectorizer.get_feature_names()[:10]

### Splitting train and test datasets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, tweets_labeled_df['sentiment'], test_size = 0.3, stratify = tweets_labeled_df['sentiment'], random_state = 1)

In [ ]:
pd.Series(y_train).value_counts(normalize = True)

In [ ]:
pd.Series(y_test).value_counts(normalize = True)

### Training a first model

In [ ]:
logistic_model = LogisticRegression(random_state = 2)

In [ ]:
logistic_model.fit(X_train, y_train)

In [ ]:
y_train_predict = logistic_model.predict(X_train)
y_test_predict = logistic_model.predict(X_test)

### Validating how this first model behaves

<img src="./imgs/confusion-matrix.png" style="height: 300px;">

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_train, y_train_predict)

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_test, y_test_predict)

<img src="./imgs/precision-recall.png" style="height: 700px;">

In [ ]:
print(precision_score(y_test, y_test_predict))
print(recall_score(y_test, y_test_predict))
print(f1_score(y_test, y_test_predict))

### Next steps

- Vectorize words using TF-IDF:

<img src="./imgs/tf-idf.png" style="height: 350px;">

- Test different algorithms: árboles de decision (random forest), suppurt vector machines, otros.
- ¿Which model is better and why?